In [24]:
import requests
import pandas as pd
from IPython.display import Image
from IPython.core.display import HTML 

In [1]:
def get_authentication():
    """
    Get authentication from Spotify API using the credentials we have 
    """
    AUTH_URL = 'https://accounts.spotify.com/api/token'
    cid ="0753888cfb694cf98b9332cd343859a3" 
    secret = "0f47a99eb4bc40eea4f3c9d6b3f1df82"
    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()
    # save the access token
    access_token = auth_response_data['access_token']

    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)}
    return headers 


In [63]:
def get_artist_id(artist: str):
    """
    Use the searching function in Spotify API to search for the most relavant artist name
    Input: String of artist name
    Output: String of artist id 
    """
    URI = 'https://api.spotify.com/v1/search?query='+ artistName +'&offset=0&limit=1&type=artist'
    response = requests.get(url = URI, headers=get_authentication())
    response = response.json()
    artist_id = response['artists']['items'][0]['id']
    return artist_id

In [91]:
def get_artist_album(artist_id: str, limit: int):
    """
    Get the album from the artist, you can specify the number of albums returned
    Input: String of artist id, Number of albums
    Output: Dataframe with relavant album information
    """
    album_url = 'https://api.spotify.com/v1/artists/' + artist_id + '/albums?limit=' + str(limit) + '&include_groups=album'
    response = requests.get(album_url, headers=get_authentication())
    response = response.json()
    df = pd.DataFrame(response['items']) # we make the data frame using the items 
    df['image_url'] = df['images'].apply(lambda x: x[0]['url']) # we need only one image url
    df['available_market_cnt'] = df['available_markets'].apply(lambda x: len(x))
    df['release_date'] = pd.to_datetime(t['release_date'] ) # change the string to datetime
    df.sort_values(by='release_date', ascending=False) # descending release time
    df = df.loc[:, ['name','id', 'release_date', 'total_tracks','image_url','available_market_cnt' ]] 
    return df 

In [5]:
artistName ='Illenium'
URI = 'https://api.spotify.com/v1/search?query='+ artistName +'&offset=0&limit=1&type=artist'
response = requests.get(url = URI, headers=get_authentication())
response = response.json()
artist_id = response['artists']['items'][0]['id']

In [69]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'


def output_dataframe(df):
    pd.set_option('display.max_colwidth', None)
    image_cols = ['image_url']  #<- define which columns will be used to convert to html
# Create the dictionariy to be passed as formatters
    format_dict = {}
    for image_col in image_cols:
        format_dict[image_col] = path_to_image_html
    display(HTML(df.to_html(escape=False ,formatters=format_dict)))


In [92]:
t = get_artist_album(artist_id, 50)
output_dataframe(t)

,name,id,release_date,total_tracks,image_url,available_market_cnt
0,Fallen Embers (Deluxe Version),0KgcBsDY5uFt4EaG3roI3x,2021-10-22,19,,184
1,Fallen Embers,4YomxhmDglPvD93W3T6l1V,2021-07-16,14,,184
2,ASCEND (Remixes),2Ds60s4LcHcb77dNQm22kA,2020-05-15,24,,184
3,ASCEND,60xcVwuQJAOyu11xf9mObS,2019-08-16,17,,182
4,Awake (Remixes),4f9voI47wfAZIp0UgwGy6o,2018-06-18,15,,184
5,Awake,5rvIgXvAPGWspXY4rDLkeU,2017-09-21,13,,184
6,Ashes (Remixes),2R7PeUrbSseLkirdasHPqg,2016-12-16,15,,184
7,Ashes,1hG9qANsJg5lYm36MYun97,2016-02-15,12,,184
